<a href="https://colab.research.google.com/github/marcgym2/MachineLearning/blob/main/Tarea9_Box-Behnken.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


## Tarea 9 (10 puntos)
- Realizar un diseño de experimentos para tu problema de estudio en la que indiques los niveles y tratamientos de interés.
- Reporta en la metodología de tu artículo el diseño de experimentos que realizaste.

In [24]:
!pip install pydoe2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pydoe2: filename=pyDOE2-1.3.0-py3-none-any.whl size=25537 sha256=e7518ab58bb6e8ea3d3c537c4c765cfb276e2da54f8f8f3160e72c615f57d15f
  Stored in directory: /root/.cache/pip/wheels/a7/56/10/2dcc175569666fb8af5c210980eb5833befd399780e71e6749
Successfully built pydoe2


In [27]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras import regularizers
import tensorflow as tf
from pyDOE2 import bbdesign

In [4]:
# Cargar dataset
from google.colab import drive
drive.mount("/content/drive")
df_measures = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/measures_v2.csv")

Mounted at /content/drive


In [28]:
# Definir variables independientes y dependientes
x_vars = ["u_q", "coolant", "ambient", "u_d", "motor_speed", "i_q", "i_d"]
y_vars = ["pm", "torque", "stator_yoke", "stator_tooth", "stator_winding"]
x_levels = [-1, 1] # Niveles codificados de las variables independientes

In [29]:
# DOE Definir el diseño óptimo de Box-Behnken
bb_design = bbdesign(len(x_vars), center=1)

In [30]:
# Transformar los valores de las variables independientes a sus valores reales
bb_design = np.multiply(bb_design, (np.max(df_measures[x_vars].values, axis=0) - 
                                    np.min(df_measures[x_vars].values, axis=0))) / 2 + \
                                    (np.max(df_measures[x_vars].values, axis=0) + 
                                     np.min(df_measures[x_vars].values, axis=0)) / 2


In [31]:
# Escalar los datos
scaler = MinMaxScaler(feature_range=(0, 1))
df_measures[x_vars + y_vars] = scaler.fit_transform(df_measures[x_vars + y_vars])


In [32]:
# Dividir los datos en conjunto de entrenamiento y prueba
train_size = int(len(df_measures) * 0.7)
train_set = df_measures.iloc[:train_size]
test_set = df_measures.iloc[train_size:]

In [33]:
# Definir el modelo de red neuronal
model = Sequential()

# Añadir las capas ocultas
model.add(Dense(128, input_dim=len(x_vars), activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))


In [34]:
# Añadir la capa de salida
model.add(Dense(len(y_vars), activation='linear', 
                kernel_regularizer=regularizers.L1L2(l1=1e-5, l2=1e-4)))


In [35]:
# Definir el optimizador y la función de pérdida
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(loss='mse', optimizer=opt)


In [39]:
# Entrenar el modelo
history = model.fit(train_set[x_vars], train_set[y_vars],
                    validation_data=(test_set[x_vars], test_set[y_vars]),
                    epochs=50, batch_size=128)


Epoch 1/50
7278/7278 [==============================] - 30s 4ms/step - loss: 0.0074 - val_loss: 0.0095
Epoch 2/50
7278/7278 [==============================] - 32s 4ms/step - loss: 0.0073 - val_loss: 0.0095
Epoch 3/50
7278/7278 [==============================] - 32s 4ms/step - loss: 0.0073 - val_loss: 0.0096
Epoch 4/50
7278/7278 [==============================] - 27s 4ms/step - loss: 0.0072 - val_loss: 0.0095
Epoch 5/50
7278/7278 [==============================] - 30s 4ms/step - loss: 0.0072 - val_loss: 0.0095
Epoch 6/50
7278/7278 [==============================] - 35s 5ms/step - loss: 0.0071 - val_loss: 0.0096
Epoch 7/50
7278/7278 [==============================] - 28s 4ms/step - loss: 0.0071 - val_loss: 0.0096
Epoch 8/50
7278/7278 [==============================] - 26s 4ms/step - loss: 0.0071 - val_loss: 0.0096
Epoch 9/50
7278/7278 [==============================] - 27s 4ms/step - loss: 0.0071 - val_loss: 0.0096
Epoch 10/50
7278/7278 [==============================] - 31s 4ms/step - l

In [40]:
# Obtener las predicciones en el conjunto de prueba
predictions = model.predict(test_set[x_vars])

12477/12477 [==============================] - 14s 1ms/step


In [41]:
# Calcular las métricas de evaluación en el conjunto de prueba
mse = np.mean(np.square(predictions - test_set[y_vars]))
rmse = np.sqrt(mse)
mae = np.mean(np.abs(predictions - test_set[y_vars]))
r2 = 1 - (np.sum(np.square(predictions - test_set[y_vars])) / 
          np.sum(np.square(test_set[y_vars] - np.mean(test_set[y_vars], axis=0))))

print("RMSE en el conjunto de prueba:", rmse)
print("R^2 en el conjunto de prueba:", r2)
print("MAE en el conjunto de prueba:", mae)
print("MSE en el conjunto de prueba:", mse)

RMSE en el conjunto de prueba: pm                0.109551
torque            0.026524
stator_yoke       0.087089
stator_tooth      0.111636
stator_winding    0.116257
dtype: float64
R^2 en el conjunto de prueba: pm                0.473920
torque            0.969545
stator_yoke       0.850503
stator_tooth      0.735437
stator_winding    0.654852
dtype: float64
MAE en el conjunto de prueba: pm                0.085861
torque            0.019625
stator_yoke       0.067936
stator_tooth      0.088294
stator_winding    0.093604
dtype: float64
MSE en el conjunto de prueba: pm                0.012001
torque            0.000704
stator_yoke       0.007585
stator_tooth      0.012463
stator_winding    0.013516
dtype: float64


/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/usr/local/lib/python3.9/dist-packages/numpy/core/fromnumeric.py:3472: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
